## 밀도 기반 군집 분석 (DBSCAN)
- 데이터가 모여 있는 밀도가 높은 공간(반경 r안에 샘플 개수 설정함)에 클러스터 레이블을 할당

In [2]:
import pandas as pd
import folium

In [4]:
file_path='./data/2016_middle_shcool_graduates_report.xlsx'
df =pd.read_excel(file_path, header=0)

print(df.info())
print(df.columns.values)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 415 entries, 0 to 414
Data columns (total 21 columns):
Unnamed: 0    415 non-null int64
지역            415 non-null object
학교명           415 non-null object
코드            415 non-null int64
유형            415 non-null object
주야            415 non-null object
남학생수          415 non-null int64
여학생수          415 non-null int64
일반고           415 non-null float64
특성화고          415 non-null float64
과학고           415 non-null float64
외고_국제고        415 non-null float64
예고_체고         415 non-null float64
마이스터고         415 non-null float64
자사고           415 non-null float64
자공고           415 non-null float64
기타진학          415 non-null float64
취업            415 non-null int64
미상            415 non-null float64
위도            415 non-null float64
경도            415 non-null float64
dtypes: float64(12), int64(5), object(4)
memory usage: 68.2+ KB
None
['Unnamed: 0' '지역' '학교명' '코드' '유형' '주야' '남학생수' '여학생수' '일반고' '특성화고' '과학고'
 '외고_국제고' '예고_체고' '마이스터고' '자사고' 

In [ ]:
#참고 url : http://python-graph-gallery.com/288-map-background-with-folium

mschool_map = folium.Map(location=[37.55, 126.98],  tiles='Stamen Terrain', zoom_start=12)
for name, lat, lng in zip (df.학교명, df.위도, df. 경도) :
    folium.CircleMarker([lat, lng],
                              radius=5,
                              color='brown',
                              fill=True,
                              fill_color='yellow',
                              fill_opacity=0.5,
                              popup=name).add_to(mschool_map)

mschool_map.save('./output/seoul_mschool_location.html')

In [8]:
#지역, 코드, 유형, 주야 등 변수의 one-hot-encoding으로 dummy 변수 변환
from sklearn import preprocessing

label_encoder = preprocessing.LabelEncoder() 
onehot_encoder = preprocessing.OneHotEncoder()

onehot_location = label_encoder.fit_transform(df['지역'])
onehot_code = label_encoder.fit_transform(df['코드'])
onehot_type = label_encoder.fit_transform(df['유형'])
onehot_day= label_encoder.fit_transform(df['주야'])

df['location'] = onehot_location
df['code'] = onehot_code
df['type'] = onehot_type
df['day'] = onehot_day

In [9]:
df.head()

,Unnamed: 0,지역,학교명,코드,유형,주야,남학생수,여학생수,일반고,특성화고,...,자공고,기타진학,취업,미상,위도,경도,location,code,type,day
0,0,성북구,서울대학교사범대학부설중학교,3,국립,주간,277,0,0.585,0.148,...,0.000,0.004,0,0.000,37.594942,127.038909,16,0,1,0
1,1,종로구,서울대학교사범대학부설여자중학교,3,국립,주간,0,256,0.680,0.199,...,0.004,0.031,0,0.000,37.577473,127.003857,22,0,1,0
2,2,강남구,개원중학교,3,공립,주간,170,152,0.817,0.047,...,0.003,0.009,0,0.003,37.491637,127.071744,0,0,0,0
3,3,강남구,개포중학교,3,공립,주간,83,72,0.755,0.097,...,0.000,0.019,0,0.000,37.480439,127.062201,0,0,0,0
4,4,서초구,경원중학교,3,공립,주간,199,212,0.669,0.017,...,0.000,0.010,0,0.000,37.510750,127.008900,14,0,0,0


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 415 entries, 0 to 414
Data columns (total 25 columns):
Unnamed: 0    415 non-null int64
지역            415 non-null object
학교명           415 non-null object
코드            415 non-null int64
유형            415 non-null object
주야            415 non-null object
남학생수          415 non-null int64
여학생수          415 non-null int64
일반고           415 non-null float64
특성화고          415 non-null float64
과학고           415 non-null float64
외고_국제고        415 non-null float64
예고_체고         415 non-null float64
마이스터고         415 non-null float64
자사고           415 non-null float64
자공고           415 non-null float64
기타진학          415 non-null float64
취업            415 non-null int64
미상            415 non-null float64
위도            415 non-null float64
경도            415 non-null float64
location      415 non-null int32
code          415 non-null int64
type          415 non-null int32
day           415 non-null int32
dtypes: float64(12), int32(3), int64(6), o

### DBSCAN 군집분석 수행

In [11]:
from sklearn import cluster

#분석에 사용할 속성 [과학고, 외고_국제고, 자사고] 진학률
column_list = [9, 10, 13]
X = df.iloc[ : , column_list]
print(X[:5])

    특성화고    과학고  마이스터고
0  0.148  0.018  0.011
1  0.199  0.000  0.000
2  0.047  0.009  0.006
3  0.097  0.013  0.019
4  0.017  0.007  0.000


### 설명변수 정규화

In [13]:
X= preprocessing.StandardScaler().fit(X).transform(X)
dbscan_model = cluster.DBSCAN(eps=0.2, min_samples=5)
dbscan_model.fit(X)

cluster_label = dbscan_model.labels_
print(cluster_label)

[-1  0 -1 -1  1  5 -1 -1  2 -1 -1 -1  2  0 -1 -1 -1 -1 -1  2 -1 -1 -1  0
  1  2 -1  1  0  0 -1 -1 -1 -1 -1  0 -1 -1 -1 -1 -1 -1 -1  0 -1 -1 -1  3
 -1 -1 -1  3 -1  1 -1 -1  4 -1 -1  4 -1 -1  4  0 -1  5  0  6  1 -1  0  1
 -1  0  8 -1 -1 -1  4 -1  3 -1 -1  3  4 -1 -1 -1  3 -1  0  2 -1  0 -1 -1
  5  0  0 -1 -1 -1 -1 -1 -1 -1 -1  0 -1 -1 -1 -1 -1 -1  1 -1 -1 -1  0 -1
 -1 -1  5  8  0 -1 -1 -1 -1 -1  7  3  0 -1  5  0  4 -1  5 -1 -1 -1 -1 -1
 -1  6 -1 -1 -1  8  4 -1 -1  6 -1 -1 -1 -1 -1  0  6 -1 -1 -1 -1 -1 -1  6
 -1 -1  7 -1 -1  6 -1  0  0 -1  3 -1  6  8 -1  6 -1 -1 -1 -1 -1 -1  5  0
  0 -1 -1 -1 -1  0  0 -1  0 -1 -1  6  0 -1 -1 -1 -1 -1  4 -1 -1  3 -1 -1
  0 -1 -1  9 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  0 -1 -1 -1  0 -1 -1 -1  9  0
  9 -1 -1 -1 -1  5  0 -1 -1 -1  0  0 -1  0 -1 -1 -1 -1 -1  0  4 -1 -1  0
 -1 -1  0  0 -1 -1  6  3 -1  6 -1 -1 -1  0 -1  0 10 -1  0  0  0 -1  0 -1
 -1 -1 -1  9 -1 -1 10  0 10  0  7  7  3  6 -1 -1 -1 -1  3  4 -1  0 -1  8
  4  8  6  0  7 -1 -1 -1 -1  0 -1 -1 -1 -1 -1  0 -1

In [19]:
df['Cluster'] = cluster_label
print(df.head())
print('\n')

   Unnamed: 0   지역               학교명  코드  유형  주야  남학생수  여학생수    일반고   특성화고  \
0           0  성북구    서울대학교사범대학부설중학교   3  국립  주간   277     0  0.585  0.148   
1           1  종로구  서울대학교사범대학부설여자중학교   3  국립  주간     0   256  0.680  0.199   
2           2  강남구             개원중학교   3  공립  주간   170   152  0.817  0.047   
3           3  강남구             개포중학교   3  공립  주간    83    72  0.755  0.097   
4           4  서초구             경원중학교   3  공립  주간   199   212  0.669  0.017   

   ...   기타진학  취업     미상         위도          경도  location  code  type  day  \
0  ...  0.004   0  0.000  37.594942  127.038909        16     0     1    0   
1  ...  0.031   0  0.000  37.577473  127.003857        22     0     1    0   
2  ...  0.009   0  0.003  37.491637  127.071744         0     0     0    0   
3  ...  0.019   0  0.000  37.480439  127.062201         0     0     0    0   
4  ...  0.010   0  0.000  37.510750  127.008900        14     0     0    0   

   Cluster  
0       -1  
1        0  
2       -1  
3       -1

### 클러스터값으로  그룹화하여 출력

In [16]:
grouped_cols =[0, 1, 3] +column_list
grouped = df.groupby('Cluster')
for key, group in grouped:
    print('* key : ' , key)
    print('* number : ' , len(group))
    print(group.iloc[:, grouped_cols].head())
    print('\n')

* key :  -1
* number :  234
   Unnamed: 0   지역  코드   특성화고    과학고  마이스터고
0           0  성북구   3  0.148  0.018  0.011
2           2  강남구   3  0.047  0.009  0.006
3           3  강남구   3  0.097  0.013  0.019
6           6  강남구   3  0.015  0.015  0.000
7           7  강남구   3  0.000  0.032  0.000


* key :  0
* number :  70
    Unnamed: 0   지역  코드   특성화고  과학고  마이스터고
1            1  종로구   3  0.199  0.0    0.0
13          13  서초구   3  0.032  0.0    0.0
23          23  강남구   3  0.025  0.0    0.0
28          28  서초구   3  0.040  0.0    0.0
29          29  강남구   3  0.051  0.0    0.0


* key :  1
* number :  7
    Unnamed: 0   지역  코드   특성화고    과학고  마이스터고
4            4  서초구   3  0.017  0.007    0.0
24          24  강남구   3  0.026  0.007    0.0
27          27  강남구   3  0.006  0.006    0.0
53          53  강동구   3  0.020  0.007    0.0
68          68  송파구   3  0.011  0.007    0.0


* key :  2
* number :  6
    Unnamed: 0   지역  코드   특성화고    과학고  마이스터고
8            8  강남구   3  0.018  0.013    0.0
12      

In [22]:
colors = {-1:'gray', 0:'coral', 1:'black', 2:'green', 3:'red', 4:'purple', 
          5:'orange', 6:'brown', 7:'magenta', 8:'yellow', 9:'brick', 10:'cyan',11:'pink'}
cluster_map = folium.Map(location=[37.55, 126.98],  tiles='Stamen Terrain', zoom_start=12)

for name, lat, lng, clus in zip (df.학교명, df.위도, df. 경도, df.Cluster) :
    folium.CircleMarker([lat, lng],
                              radius=5,
                              color=colors[clus],
                              fill=True,
                              fill_color=colors[clus] ,
                              fill_opacity=0.5,
                              popup=name).add_to(cluster_map)

cluster_map.save('./output/seoul_mschool_cluster.html')
